In [1]:
from pprint import pprint
import binance as bb
import itertools as it
import copy

In [2]:
def get_books(pairs, limit=100):
    books = {}
    for pair in pairs:
        books[pair] = bb.fetch_order_book(bb.API_KEY, pair, limit=limit)
    return books


def get_best_price(orders, money_in, inverse=False):
    # What happen if order book too small?
    avl = money_in
    money_out = 0
    for price, amount in orders:
        price, amount = float(price), float(amount)
        diff = avl - (amount if not inverse else amount*price)
        if diff <= 0:
            money_out += avl*price if not inverse else avl/price
            break
        else:
            money_out += (price*amount) if not inverse else amount
            avl -= amount if not inverse else amount*price
    return money_out/money_in

In [3]:
eth_pairs = ['ETHEUR', 'ETHUSDT', 'ETHBUSD', 'ETHUSDC','ETHTUSD']
usdt_pairs = ['EURUSDT', 'BUSDUSDT', 'TUSDUSDT', 'USDCUSDT']

eth_books = get_books(eth_pairs, limit=10)
usdt_books = get_books(usdt_pairs, limit=10)

In [4]:
# Normalize_books
normalized_books = copy.deepcopy(eth_books)
for eth_pair in eth_books:
    if eth_pair == 'ETHUSDT': continue
    usdt_pair = [pair for pair in usdt_pairs if eth_pair[3:] in pair][0]
    buy = lambda x: x * get_best_price(usdt_books[usdt_pair]['asks'], x)  # Buying EUR/BUSD/USDC/TUSD
    sell = lambda x: x * get_best_price(usdt_books[usdt_pair]['bids'], x)  # Selling EUR/BUSD/USDC/TUSD
    normalized_books[eth_pair]['asks'] = [[buy(float(price)), amount] for price, amount in normalized_books[eth_pair]['asks']]
    normalized_books[eth_pair]['bids'] = [[sell(float(price)), amount] for price, amount in normalized_books[eth_pair]['bids']]

In [7]:
amount = 100
fees = 0.075
best_prices = {'asks': [], 'bids': []}
for pair in normalized_books:
    ask_price = 1/get_best_price(normalized_books[pair]['asks'], amount, inverse=1)
    bid_price = 1/get_best_price(normalized_books[pair]['bids'], amount, inverse=1)
    best_prices['asks'] += [(pair, ask_price)]
    best_prices['bids'] += [(pair, bid_price)]
best_ask = min(best_prices['asks'], key=lambda x: x[1])
best_bid = max(best_prices['bids'], key=lambda x: x[1])
pairs = f"{best_ask[0]}/{best_bid[0]}"
profit = (best_bid[1]-best_ask[1])

print(pairs, profit)
pprint(best_prices)

ETHUSDT/ETHUSDT -0.030000000000001137
{'asks': [('ETHEUR', 205.97528335445892),
          ('ETHUSDT', 205.25),
          ('ETHBUSD', 205.31699364821725),
          ('ETHUSDC', 205.34801237574177),
          ('ETHTUSD', 205.3701667666522)],
 'bids': [('ETHEUR', 204.122076),
          ('ETHUSDT', 205.22),
          ('ETHBUSD', 205.11507324283906),
          ('ETHUSDC', 205.192902),
          ('ETHTUSD', 205.16237)]}


In [6]:
fee = 0.00075
ask_asset = best_ask[0][3:]
bid_asset = best_bid[0][3:]

msg = ""

msg += "\t" + f"HOLDING {amount} USDT" + "\n"

# Convert if starting asset is not in USDT
if ask_asset != 'USDT':
    ask_asset_book = usdt_books[ask_asset + 'USDT']['asks']
    buy_amount = get_best_price(ask_asset_book, amount, inverse=1) * amount
    msg += "\t" + f"BUY {buy_amount} {ask_asset + 'USDT'}" + "\n"
    holding = buy_amount*(1-fee)
    msg += "\t" + f"HOLDING {holding} {ask_asset}" + "\n"
else:
    holding = amount

# Buy ETH
buy_amount = holding*get_best_price(eth_books[best_ask[0]]['asks'], holding, inverse=1)
holding = buy_amount*(1-fee)
msg += "\t" + f"BUY {buy_amount} {best_ask[0]}" + "\n"
msg += "\t" + f"HOLDING {holding} ETH" + "\n"

# Sell ETH
sell_amount = holding
msg += "\t" + f"SELL {sell_amount} {best_bid[0]}" + "\n"
holding = sell_amount*get_best_price(eth_books[best_bid[0]]['bids'], sell_amount)*(1-fee)
msg += "\t" + f"HOLDING {holding} {bid_asset}" + "\n"

# If asset is not in USDT convert it
if bid_asset != 'USDT':
    msg += f"SELL {holding} {bid_asset + 'USDT'}" + "\n"
    bid_asset_book = usdt_books[bid_asset + 'USDT']['bids']
    holding = get_best_price(bid_asset_book, holding) * holding * (1-fee)

msg += "\t" + f"HOLDING: {holding}" + "\n"
msg += "\t" + f"PROFIT: {holding - amount}"

print(msg)

	HOLDING 100 USDT
	BUY 0.48721071863580995 ETHUSDT
	HOLDING 0.48684531059683306 ETH
	SELL 0.48684531059683306 ETHUSDT
	HOLDING 99.83546184470157 USDT
	HOLDING: 99.83546184470157
	PROFIT: -0.1645381552984304
